In [1]:
import unicom
import torch
from torchsummary import summary
import torchvision

In [5]:
temp = torch.load('./checkpoints/UNICOM_ViT_B_32_based.pth')['state_dict']
state_dict = {}
for k, v in temp.items():
    if 'encoder_q' in k:
        state_dict['.'.join(k.split('.')[1:])] = v

In [10]:
temp.keys()

odict_keys(['queue', 'queue_ptr', 'encoder_q.pos_embed', 'encoder_q.patch_embed.proj.weight', 'encoder_q.patch_embed.proj.bias', 'encoder_q.blocks.0.norm1.weight', 'encoder_q.blocks.0.norm1.bias', 'encoder_q.blocks.0.norm2.weight', 'encoder_q.blocks.0.norm2.bias', 'encoder_q.blocks.0.attn.qkv.weight', 'encoder_q.blocks.0.attn.proj.weight', 'encoder_q.blocks.0.attn.proj.bias', 'encoder_q.blocks.0.mlp.fc1.weight', 'encoder_q.blocks.0.mlp.fc1.bias', 'encoder_q.blocks.0.mlp.fc2.weight', 'encoder_q.blocks.0.mlp.fc2.bias', 'encoder_q.blocks.1.norm1.weight', 'encoder_q.blocks.1.norm1.bias', 'encoder_q.blocks.1.norm2.weight', 'encoder_q.blocks.1.norm2.bias', 'encoder_q.blocks.1.attn.qkv.weight', 'encoder_q.blocks.1.attn.proj.weight', 'encoder_q.blocks.1.attn.proj.bias', 'encoder_q.blocks.1.mlp.fc1.weight', 'encoder_q.blocks.1.mlp.fc1.bias', 'encoder_q.blocks.1.mlp.fc2.weight', 'encoder_q.blocks.1.mlp.fc2.bias', 'encoder_q.blocks.2.norm1.weight', 'encoder_q.blocks.2.norm1.bias', 'encoder_q.bloc

In [9]:
state_dict.keys()

dict_keys(['pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.3.norm1.weight', 'blocks.3.norm1.bias', 'blocks.3.norm2.weight', 'blocks.3.norm2.bias', 'blocks.3.attn.qkv.weight', 'blocks.3.attn.proj.weight', 'blocks.3.attn.proj.bias', 'blocks.4.norm1.weight', 'blocks.4.norm1.bias', 'blocks.4.norm2.weight', 'blocks.4.norm2.bias', 'blocks.4.attn.qkv.weight', 'blocks.4.attn.proj.weight', 'blocks.4.attn.proj.bias', 'blocks.5.norm1.weight',

In [2]:
model = unicom.load('ViT-B/32')[0]
model.load_state_dict(state_dict)

NameError: name 'state_dict' is not defined

In [11]:

inputs = torch.nn.Parameter(torch.ones(4))

In [13]:
inputs[0].item()

1.0

In [2]:
unicom.available_models()

['ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']

In [3]:
model = unicom.load('ViT-B/32')[0]

In [20]:
from torch import nn
avgpool = nn.AdaptiveAvgPool1d(1)

In [8]:
model.blocks[len(model.blocks) - 1]

Block(
  (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): Attention(
    (qkv): Linear(in_features=768, out_features=2304, bias=False)
    (proj): Linear(in_features=768, out_features=768, bias=True)
  )
  (drop_path): DropPath(drop_prob=0.100)
  (mlp): Mlp(
    (fc1): Linear(in_features=768, out_features=3072, bias=True)
    (act): ReLU6()
    (fc2): Linear(in_features=3072, out_features=768, bias=True)
  )
)

In [25]:
torch.manual_seed(42)

In [27]:
inputs = torch.randn((32, 3, 224, 224))
with torch.no_grad():
    x1 = model(inputs)

/home/gp.sc.cc.tohoku.ac.jp/duanct/miniconda3/envs/py2cuda118/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [30]:
model.eval()
x = model.patch_embed(inputs)
x = x + model.pos_embed
for block in model.blocks:
    x = block(x)
    # print(x.shape)
x = model.norm(x)
# print(x.shape)
x = x.reshape(32, -1)
x = model.feature(x)
# print(x.shape)


print(torch.allclose(x, x1))

False


In [24]:
torch.any(x == x1)

tensor(False)

In [12]:

x == x1

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [25]:
avgpool(x.transpose(1, 2)).squeeze(-1).shape

torch.Size([32, 768])

In [4]:
inputs = torch.randn((32, 3, 224, 224))
model = unicom.load('ViT-B/32')[0]
model = model.cuda()
summary(model, (3, 224, 224), batch_size=2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [2, 768, 7, 7]       2,360,064
    PatchEmbedding-2               [2, 49, 768]               0
         LayerNorm-3               [2, 49, 768]           1,536
            Linear-4              [2, 49, 2304]       1,769,472
            Linear-5               [2, 49, 768]         590,592
         Attention-6               [2, 49, 768]               0
          Identity-7               [2, 49, 768]               0
         LayerNorm-8               [2, 49, 768]           1,536
            Linear-9              [2, 49, 3072]       2,362,368
            ReLU6-10              [2, 49, 3072]               0
           Linear-11               [2, 49, 768]       2,360,064
              Mlp-12               [2, 49, 768]               0
         Identity-13               [2, 49, 768]               0
            Block-14               [2, 

In [10]:
model = torchvision.models.resnet50()
model = model.cuda()
summary(model, (3, 224, 224), batch_size=2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [2, 64, 112, 112]           9,408
       BatchNorm2d-2          [2, 64, 112, 112]             128
              ReLU-3          [2, 64, 112, 112]               0
         MaxPool2d-4            [2, 64, 56, 56]               0
            Conv2d-5            [2, 64, 56, 56]           4,096
       BatchNorm2d-6            [2, 64, 56, 56]             128
              ReLU-7            [2, 64, 56, 56]               0
            Conv2d-8            [2, 64, 56, 56]          36,864
       BatchNorm2d-9            [2, 64, 56, 56]             128
             ReLU-10            [2, 64, 56, 56]               0
           Conv2d-11           [2, 256, 56, 56]          16,384
      BatchNorm2d-12           [2, 256, 56, 56]             512
           Conv2d-13           [2, 256, 56, 56]          16,384
      BatchNorm2d-14           [2, 256,